In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library



In [2]:
df = pd.read_csv("toronto_neighboorhood_coordinate_data.csv")

In [3]:
print("There are {} unique Borough in Dataset".format(df.Borough.nunique()))

There are 10 unique Borough in Dataset


In [4]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### We only work with borough which contain word toronto.

In [5]:
df1=df[df.Borough.astype('str').str.contains('Toronto')].copy()

In [6]:
df1.reset_index(inplace=True)

In [7]:
df1.drop(columns=['index','Postal Code'],inplace=True)

In [8]:
print("There are {} unique Borough in Dataset".format(df1.Borough.nunique()))

There are 4 unique Borough in Dataset


#### This Dataset slight different than New York dataset

In [9]:
toronto_neighborhood = df1.copy()

#### Let's get the Geographical Coordinate of Toronto,Canada City 

In [10]:
address = 'Toronto, CANADA'

geolocator = Nominatim(user_agent="location-explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinate of Toronto,Canada City are {}, {}.'.format(latitude, longitude))



The coordinate of Toronto,Canada City are 43.6534817, -79.3839347.


####  Create a map of Toronto with neighborhoods superimposed on top.

In [11]:
toronto_map = folium.Map(location=[latitude,longitude], zoom_start=11)

#add neighborhood

for lat, lan, borough, nei in zip(toronto_neighborhood['Latitude'],toronto_neighborhood['Longitude'],toronto_neighborhood['Borough'],toronto_neighborhood['Neighborhood']):
    label = '{},{}'.format(nei, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lan],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='blue',
        fill_opacity=0.8,
        parse_html=False).add_to(toronto_map)
    
toronto_map

In [12]:
toronto_neighborhood.Borough.value_counts()

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [13]:
downtown_data = toronto_neighborhood.loc[toronto_neighborhood.Borough=="Downtown Toronto",:]
downtown_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
5,Downtown Toronto,Berczy Park,43.644771,-79.373306


#### Let's get the Geographical Coordinate of Downtown Toronto.

In [14]:
address = 'Downtown Toronto, Toronto, CANADA'

geolocator = Nominatim(user_agent="location-explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


#### Let's visualize  Downtown Toronto and it's neighborhood in it.

In [15]:
downtown_map = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lan, nei in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lan],
    radius=5,
    fill=True,
    color='black',
    fill_color='blue',
    popup=label,
    fill_opacity=0.8,
    parse_html=False).add_to(downtown_map)

downtown_map

#### Foursqure Credential and latest version

In [44]:
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret'
VERSION = 'version'



print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('LATEST_VERSION: ' + VERSION)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret
LATEST_VERSION: version


#### Let's explore the one of the neighborhood in our dataframe.

First get Neighborhood Name and then Latitude & Longitude

In [17]:
neighborhood_name = downtown_data.loc[7,'Neighborhood']
neighborhood_latitude = downtown_data.loc[7,'Latitude']
neighborhood_longitude = downtown_data.loc[7,'Longitude']
LIMIT = 50
radius = 500

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude,neighborhood_longitude))


Latitude and longitude values of Christie are 43.669542, -79.4225637.


#### Now, let's get the top 50 venues that are in Christie within a radius of 500 meters.
First create url and then send Get request 

In [18]:
url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,CLIENT_SECRET,VERSION,neighborhood_latitude,neighborhood_longitude,radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=YG52G4CEK40PDNSPX4321TYHPVE5GDHW5GTCH0ZHFIY0YU2A&client_secret=LGA1AA31JNJLXBXAEKRIWY52FYJFQIFB2Y3ATERXLRG3ESSQ&v=20200514&ll=43.669542,-79.4225637&radius=500&limit=50'

In [21]:
venue_data = requests.get(url).json()
venue_data    #now, we get our desire result

{'meta': {'code': 200, 'requestId': '5ebe9592c94979001b026c53'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Christie Pits',
  'headerFullLocation': 'Christie Pits, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 16,
  'suggestedBounds': {'ne': {'lat': 43.6740420045, 'lng': -79.41635411972038},
   'sw': {'lat': 43.6650419955, 'lng': -79.42877328027961}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adcfd7cf964a5203e6321e3',
       'name': 'Fiesta Farms',
       'location': {'address': '200 Christie St',
        'crossStreet': 'at Essex St',
        'lat': 43.66847077052224,
        'lng': -79.42048512748114,
        'labeledLatLngs': [{'label': 'd

In [22]:
#now define function which get category name

def get_category_name(row):
    try:
        category_list_dict = row['categories']
    except:
        category_list_dict = row['venue.categories']
    if len(category_list_dict) == 0:
        return None
    else:
        return category_list_dict[0]['name']   


In [23]:
venues = venue_data['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)   #we get our data in pandas dataframe

# nearby_venues.head()  #examine dataframe

necessary_cols = ['venue.name', 'venue.location.lat', 'venue.location.lng', 'venue.categories']

nearby_venues = nearby_venues.loc[:,necessary_cols]
#apply our function
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_name,axis=1)
#modify column names so it can be easily use and read
nearby_venues.columns =[ col_name.split('.')[-1] for col_name in nearby_venues.columns]




C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
print('We got {} venues which were returned by Foursquare Api'.format(nearby_venues.shape[0]))

We got 16 venues which were returned by Foursquare Api


In [25]:
nearby_venues.tail()

,name,lat,lng,categories
11,Marlenes Just Babies,43.671824,-79.420499,Baby Store
12,Dupont Disco,43.670490,-79.426611,Nightclub
13,Marian Engel Park,43.673754,-79.423988,Park
14,Garrison Creek Park,43.671690,-79.427805,Park
15,Foto Grocery,43.667979,-79.428217,Grocery Store


In [26]:
downtown_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
5,Downtown Toronto,Berczy Park,43.644771,-79.373306


##  Explore Neighborhoods in Downtown Toronto

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood Name', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
downtown_all_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude'])

In [29]:
print(downtown_all_venues.shape)
downtown_all_venues.head()

(799, 7)


,Neighborhood Name,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


#### Check how many venues were returned for each neighboorhood

In [30]:
downtown_all_venues.groupby('Neighborhood Name').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood Name,,,,,,
Berczy Park,50,50,50,50,50,50
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,50,50,50,50,50,50
Christie,16,16,16,16,16,16
Church and Wellesley,50,50,50,50,50,50
"Commerce Court, Victoria Hotel",50,50,50,50,50,50
"First Canadian Place, Underground city",50,50,50,50,50,50
"Garden District, Ryerson",50,50,50,50,50,50
"Harbourfront East, Union Station, Toronto Islands",50,50,50,50,50,50


In [31]:
print("There are {} unique categories in dataframe".format(downtown_all_venues['Venue Category'].nunique()))

There are 182 unique categories in dataframe


## Analyze each Neighborhood

In [32]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_all_venues['Venue Category'])
# add neighborhood name column back to dataframe

#here you have care about Category names because we have one category whose name is also Neighborhood
downtown_onehot['Neighborhood Name'] = downtown_all_venues['Neighborhood Name'] 

# downtown_onehot = pd.concat([downtown_onehot,downtown_all_venues['Neighborhood']],axis=1)  #another way

# downtown_onehot.shape

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()


,Neighborhood Name,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hobby Shop,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [33]:
downtown_onehot.shape          #new shape of dataframe

(799, 183)

In [34]:
downtown_group = downtown_onehot.groupby('Neighborhood Name').mean().reset_index()
downtown_group.head()


,Neighborhood Name,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hobby Shop,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,0.02,0.00,0.00,0.0,0.0,0.0000,0.02,0.04,0.0,0.00,0.0,0.02,0.02,0.0,0.04,0.0,0.0,0.02,0.0000,0.00,0.0000,0.0,0.02,0.0,0.00,0.00,0.00,0.0,0.0400,0.0000,0.00,0.04,0.0,0.0,0.02,0.06,0.0800,0.0,0.0,0.0,0.0,0.0,0.02,0.00,0.02,0.0,0.02,0.02,0.00,0.0,0.00,0.0,0.0200,0.00,0.00,0.00,0.0,0.00,0.02,0.0,0.0,0.00,0.02,0.0,0.0,0.02,0.0,0.0,0.02,0.02,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.02,0.02,0.02,0.00,0.00,0.0000,0.0,0.0,0.00,0.0,0.02,0.0,0.0,0.00,0.00,0.0000,0.02,0.02,0.0,0.00,0.0,0.02,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0000,0.0,0.00,0.0,0.0,0.0,0.0,0.020,0.0,0.0,0.0,0.00,0.0000,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0000,0.0400,0.0,0.00,0.00,0.00,0.00,0.0,0.0000,0.04,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.02,0.0,0.02,0.0,0.00,0.02,0.0,0.0,0.0,0.02,0.0,0.00,0.0,0.0,0.02,0.0,0.0,0.0,0.00,0.00
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0000,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0625,0.00,0.0625,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0000,0.0000,0.00,0.00,0.0,0.0,0.00,0.00,0.0000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0000,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0625,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0000,0.00,0.00

In [ ]:
#### Let's print each neighborhood along with the top 5 most common venues

In [35]:
num_top_venues = 5

for name in downtown_group['Neighborhood Name']:
    print("----"+name+"----")
    temp = downtown_group[downtown_group['Neighborhood Name'] == name].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]     #we don't need first row 
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values(by='freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.06
2            Beer Bar  0.04
3  Seafood Restaurant  0.04
4                Café  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4     Boat or Ferry  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1      Sandwich Place  0.06
2  Italian Restaurant  0.06
3                Café  0.06
4        Burger Joint  0.04


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3    Candy Store  0.06
4    Coffee Shop  0.06


----Church and Wellesley----
              venue  freq
0  Sushi Restaurant  0.08
1        Restaurant  0.06
2       Coffee Shop  0.06
3       Yoga Studio  0.04
4         Gastropub  0.0

In [36]:
def return_most_common_venues(row, num_top_venues):
    
    row_categories = row.iloc[1:]
#     print(row_categories)
    row_categories_sorted = row_categories.sort_values(ascending=False)
#     print(row_categories_sorted)   #like Coffe shop ---- 0.08, hotel -- 0.07
    return row_categories_sorted.index[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
cols = ['Neighborhood Name']
for ind in np.arange(num_top_venues):
    try:
        cols.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        cols.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=cols)
neighborhoods_venues_sorted['Neighborhood Name'] = downtown_group['Neighborhood Name']

for ind in np.arange(downtown_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Restaurant,Café,Cheese Shop,Bakery,Beer Bar,Grocery Store,Cosmetics Shop
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Harbor / Marina,Boutique,Rental Car Location,Boat or Ferry,Plane,Sculpture Garden
2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Park,Middle Eastern Restaurant,Gastropub
3,Christie,Grocery Store,Café,Park,Italian Restaurant,Coffee Shop,Diner,Nightclub,Candy Store,Restaurant,Baby Store
4,Church and Wellesley,Sushi Restaurant,Coffee Shop,Restaurant,Yoga Studio,Gastropub,Men's Store,Japanese Restaurant,Indian Restaurant,Sake Bar,Bookstore


## Cluster Neighborhoods

In [38]:
# set number of clusters
kclusters = 5

downtown_group_clustering = downtown_group.drop('Neighborhood Name', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1)
#fit the model
kmeans.fit(downtown_group_clustering)
# check cluster labels generated for each row in the dataframe
cluster_label = kmeans.labels_[0:10] 
print(cluster_label)
cluster_centroid = kmeans.cluster_centers_
print(len(cluster_centroid))
# cluster_centroid            


[0 3 2 4 0 0 0 0 0 0]
5


#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)  #we add a new col called Cluster Labels

downtown_data_merged = downtown_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood

downtown_data_merged = pd.merge(downtown_data_merged,neighborhoods_venues_sorted, left_on='Neighborhood', right_on='Neighborhood Name')
downtown_data_merged.drop(["Neighborhood Name"],axis=1,inplace=True)  #we have two nighborhood cols
downtown_data_merged.head() # check the last columns!


# downtown_data_merged['Cluster Labels'].value_counts() --- how many neighborhood in each cluster

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Bakery,Park,Breakfast Spot,Pub,Café,Theater,Yoga Studio,Mexican Restaurant,Restaurant
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,Sandwich Place,Juice Bar,Mexican Restaurant,Burrito Place,Beer Bar,Japanese Restaurant
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Café,Middle Eastern Restaurant,Coffee Shop,Restaurant,Ramen Restaurant,Clothing Store,Theater,Bookstore,Tea Room,Cosmetics Shop
3,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Gastropub,Coffee Shop,American Restaurant,Cocktail Bar,Cosmetics Shop,Creperie,Seafood Restaurant,Fish Market,Farmers Market
4,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Restaurant,Café,Cheese Shop,Bakery,Beer Bar,Grocery Store,Cosmetics Shop


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
# print(x)
colors_array = cm.rainbow(np.linspace(0, 1, 5))
# print(colors_array)
rainbow = [colors.rgb2hex(i) for i in colors_array]
# print(rainbow)
# add markers to the map
markers_colors = []
for lat, lan, name, cluster in zip(downtown_data_merged['Latitude'], downtown_data_merged['Longitude'], downtown_data_merged['Neighborhood'], downtown_data_merged['Cluster Labels']):
    label = folium.Popup(str(name) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lan],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


## Examine Clusters

#### cluser with label 1

In [41]:
downtown_data_merged.loc[downtown_data_merged['Cluster Labels'] == 1, downtown_data_merged.columns[[1] + list(range(5,  downtown_data_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Rosedale,Park,Playground,Trail,Yoga Studio,Cosmetics Shop,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store


#### cluser with label 0

In [42]:
downtown_data_merged.loc[downtown_data_merged['Cluster Labels'] == 0, downtown_data_merged.columns[[1] + list(range(5,  downtown_data_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Garden District, Ryerson",Café,Middle Eastern Restaurant,Coffee Shop,Restaurant,Ramen Restaurant,Clothing Store,Theater,Bookstore,Tea Room,Cosmetics Shop
3,St. James Town,Café,Gastropub,Coffee Shop,American Restaurant,Cocktail Bar,Cosmetics Shop,Creperie,Seafood Restaurant,Fish Market,Farmers Market
4,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Restaurant,Café,Cheese Shop,Bakery,Beer Bar,Grocery Store,Cosmetics Shop
7,"Richmond, Adelaide, King",Coffee Shop,Café,Pizza Place,Restaurant,Steakhouse,American Restaurant,Concert Hall,Seafood Restaurant,Opera House,Smoke Shop
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Hotel,Café,Park,Plaza,Brewery,Fried Chicken Joint,Bubble Tea Shop,Beer Bar
9,"Toronto Dominion Centre, Design Exchange",Café,Coffee Shop,Seafood Restaurant,Gastropub,Restaurant,Hotel,Japanese Restaurant,Ice Cream Shop,Pub,Speakeasy
10,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Hotel,Gym,Restaurant,Seafood Restaurant,Beer Bar,Gastropub,Japanese Restaurant,Deli / Bodega
11,"University of Toronto, Harbord",Café,Bakery,Bar,Italian Restaurant,Japanese Restaurant,Bookstore,Restaurant,Beer Bar,Beer Store,Sandwich Place
12,"Kensington Market, Chinatown, Grange Park",Café,Bakery,Coffee Shop,Mexican Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Gaming Cafe,Dessert Shop,Poutine Place,Burger Joint
15,Stn A PO Boxes,Coffee Shop,Café,Beer Bar,Cheese Shop,Hotel,Bakery,Restaurant,Creperie,Seafood Restaurant,Cocktail Bar
